In [1]:
import pandas as pd
#import requests 
#import zipfile
#import sqlalchemy
import psycopg2

import _functions_sql as fs
import _functions_data_files as fdf

In [15]:
# download & extract
searched_file = 'Emissions_crops_E_All_Data_(Normalized).csv'

zipfile = fdf.download_file(
    'https://fenixservices.fao.org/faostat/static/bulkdownloads/Emissions_crops_E_All_Data_(Normalized).zip'
)
if zipfile:
    extracted_files = fdf.unzip_file(zipfile)
    if searched_file in extracted_files:
        csv_file = fdf.get_file_path(searched_file, check_exist=True)
        if csv_file:
            print('file found: \'{0}\''.format(csv_file))


#csv_file = fdf.get_file_path('Emissions_crops_E_All_Data_(Normalized).csvp', check_exist=True)
#if not csv_file: print('ERROR! download or extraction failed')

file downloaded: Emissions_crops_E_All_Data_(Normalized).zip
files extracted: ['Emissions_crops_E_All_Data_(Normalized).csv', 'Emissions_crops_E_AreaCodes.csv', 'Emissions_crops_E_Flags.csv', 'Emissions_crops_E_ItemCodes.csv']
file found: './data/Emissions_crops_E_All_Data_(Normalized).csv'


In [3]:
# Import crops emissions data for all countries
emissions_crops = pd.read_csv(path+csv_file, encoding='latin-1', low_memory=False)

forced_conversions = {
    'Note': str # handle DtypeWarning:
                # (missing values are now read as empty string)
}
#emissions_crops = pd.read_csv(path+csv_file, encoding='latin-1', converters=forced_conversions)

NameError: name 'path' is not defined

In [62]:
emissions_crops.head(10)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Source Code,Source,Unit,Value,Flag,Note
0,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1961,1961,3050,FAO TIER 1,kt,0.1141,E,
1,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1962,1962,3050,FAO TIER 1,kt,0.1141,E,
2,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1963,1963,3050,FAO TIER 1,kt,0.1141,E,
3,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1964,1964,3050,FAO TIER 1,kt,0.1145,E,
4,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1965,1965,3050,FAO TIER 1,kt,0.1145,E,
5,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1966,1966,3050,FAO TIER 1,kt,0.1135,E,
6,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1967,1967,3050,FAO TIER 1,kt,0.1062,E,
7,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1968,1968,3050,FAO TIER 1,kt,0.1071,E,
8,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1969,1969,3050,FAO TIER 1,kt,0.1079,E,
9,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1970,1970,3050,FAO TIER 1,kt,0.1087,E,


In [61]:
emissions_crops["Unit"].unique()

array(['kt', 'kg', 't', 'ha'], dtype=object)

In [52]:
emissions_crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891492 entries, 0 to 891491
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Area Code        891492 non-null  int64  
 1   Area Code (M49)  891492 non-null  object 
 2   Area             891492 non-null  object 
 3   Item Code        891492 non-null  int64  
 4   Item Code (CPC)  891492 non-null  object 
 5   Item             891492 non-null  object 
 6   Element Code     891492 non-null  int64  
 7   Element          891492 non-null  object 
 8   Year Code        891492 non-null  int64  
 9   Year             891492 non-null  int64  
 10  Source Code      891492 non-null  int64  
 11  Source           891492 non-null  object 
 12  Unit             891492 non-null  object 
 13  Value            891492 non-null  float64
 14  Flag             891492 non-null  object 
 15  Note             891492 non-null  object 
dtypes: float64(1), int64(6), object(9)
mem

In [7]:
engine = sf.get_engine()
schema = 'capstone_envirolytics'
table_name = 'fao_emissions_crops_sh-test'

In [8]:
# Export the table


if engine!=None:
    try:
        emissions_crops.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schema that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

: 